# Data Source

https://github.com/CSSEGISandData/COVID-19

# Libraries

In [390]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime, timedelta
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# to download data
import wget
# numerical analysis
import numpy as np

# Downloading data

In [391]:
# remove all existing csv files
! rm *.csv

# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv',
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']

# download files
for url in urls:
    filename = wget.download(url)

# Dataframes

In [392]:
# dataset
# ======

conf_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('time_series_covid19_deaths_global.csv')
recv_df = pd.read_csv('time_series_covid19_recovered_global.csv')

In [393]:
# conf_df.head()
# deaths_df.head()
# recv_df.head()

In [394]:
# conf_df.columns
# deaths_df.columns
# recv_df.columns

In [395]:
# conf_df.columns[4:]
# deaths_df.columns[4:]
# recv_df.columns[4:]

# Merging dataframes

In [396]:
# extract dates
dates = conf_df.columns[4:]

# melt dataframes into longer format
# ==================================
conf_df_long = conf_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Confirmed')

deaths_df_long = deaths_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Deaths')

recv_df_long = recv_df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=dates, var_name='Date', value_name='Recovered')

recv_df_long = recv_df_long[recv_df_long['Country/Region']!='Canada']

print(conf_df_long.shape)
print(deaths_df_long.shape)
print(recv_df_long.shape)

(53732, 6)
(53732, 6)
(50904, 6)


In [397]:
# merge dataframes
# ================

full_table = pd.merge(left=conf_df_long, right=deaths_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])
full_table = pd.merge(left=full_table, right=recv_df_long, how='left',
                      on=['Province/State', 'Country/Region', 'Date', 'Lat', 'Long'])

full_table.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0,0,0.0
1,NaN,Albania,41.15330,20.168300,1/22/20,0,0,0.0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0,0,0.0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0,0,0.0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0,0,0.0


# Preprocessing

In [398]:
# Convert to proper date format
full_table['Date'] = pd.to_datetime(full_table['Date'])

# fill na with 0
full_table['Recovered'] = full_table['Recovered'].fillna(0)

# convert to int datatype
full_table['Recovered'] = full_table['Recovered'].astype('int')

In [399]:
# fixing Country names
# ====================

# renaming countries, regions, provinces
full_table['Country/Region'] = full_table['Country/Region'].replace('Korea, South', 'South Korea')

# Greenland
full_table.loc[full_table['Province/State']=='Greenland', 'Country/Region'] = 'Greenland'

# Mainland china to China
full_table['Country/Region'] = full_table['Country/Region'].replace('Mainland China', 'China')

In [400]:
# removing
# =======

# removing canada's recovered values
full_table = full_table[full_table['Province/State'].str.contains('Recovered')!=True]

# removing county wise data to avoid double counting
full_table = full_table[full_table['Province/State'].str.contains(',')!=True]

In [401]:
# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

# filling missing values 
# ======================
# fill missing province/state value with ''
full_table[['Province/State']] = full_table[['Province/State']].fillna('')
# fill missing numerical values with 0
cols = ['Confirmed', 'Deaths', 'Recovered', 'Active']
full_table[cols] = full_table[cols].fillna(0)

# fixing datatypes
full_table['Recovered'] = full_table['Recovered'].astype(int)

# random rows
full_table.sample(6)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
33474,,Uruguay,-32.522800,-55.765800,2020-05-26,789,22,638,129
19608,,Saint Vincent and the Grenadines,12.984300,-61.287200,2020-04-04,7,0,1,6
37723,Bermuda,United Kingdom,32.307800,-64.750500,2020-06-11,141,9,127,5
26280,,Tunisia,33.886917,9.537499,2020-04-29,980,40,294,646
36436,,Western Sahara,24.215500,-12.885800,2020-06-06,9,1,6,2
3092,Aruba,Netherlands,12.521100,-69.968300,2020-02-02,0,0,0,0


# Fixing off data

In [402]:
# new values
feb_12_conf = {'Hubei' : 34874}

In [403]:
# function to change value
def change_val(date, ref_col, val_col, dtnry):
    for key, val in dtnry.items():
        full_table.loc[(full_table['Date']==date) & (full_table[ref_col]==key), val_col] = val

In [404]:
# changing values
change_val('2/12/20', 'Province/State', 'Confirmed', feb_12_conf)

In [405]:
# checking values
full_table[(full_table['Date']=='2/12/20') & (full_table['Province/State']=='Hubei')]

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active
5648,Hubei,China,30.9756,112.2707,2020-02-12,34874,1068,2686,29612


In [406]:
# Ship
# ====

# ship rows containing ships with COVID-19 reported cases
ship_rows = full_table['Province/State'].str.contains('Grand Princess') | \
            full_table['Province/State'].str.contains('Diamond Princess') | \
            full_table['Country/Region'].str.contains('Diamond Princess') | \
            full_table['Country/Region'].str.contains('MS Zaandam')

# ship
ship = full_table[ship_rows]

# Latest cases from the ships
ship_latest = ship[ship['Date']==max(ship['Date'])]
# ship_latest.style.background_gradient(cmap='Pastel1_r')

# skipping rows with ships info
full_table = full_table[~(ship_rows)]

# WHO Region

https://en.wikipedia.org/wiki/WHO_regions

In [407]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [408]:
# add 'WHO Region' column
full_table['WHO Region'] = full_table['Country/Region'].map(who_region)

# find missing values
full_table[full_table['WHO Region'].isna()]['Country/Region'].unique()

array([], dtype=object)

In [409]:
# Cleaning data
# =============

# fixing Country values
full_table.loc[full_table['Province/State']=='Greenland', 'Country/Region'] = 'Greenland'

# Active Case = confirmed - deaths - recovered
full_table['Active'] = full_table['Confirmed'] - full_table['Deaths'] - full_table['Recovered']

# replacing Mainland china with just China
full_table['Country/Region'] = full_table['Country/Region'].replace('Mainland China', 'China')

# filling missing values 
full_table[['Province/State']] = full_table[['Province/State']].fillna('')
full_table[['Confirmed', 'Deaths', 'Recovered', 'Active']] = full_table[['Confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)

# fixing datatypes
full_table['Recovered'] = full_table['Recovered'].astype(int)

full_table.sample(6)

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
41845,,Congo (Brazzaville),-0.228000,15.827700,2020-06-27,1087,37,456,594,Africa
33212,,Vietnam,14.058324,108.277199,2020-05-25,326,0,272,54,Western Pacific
42090,Hubei,China,30.975600,112.270700,2020-06-28,68135,4512,63623,0,Western Pacific
45008,Guangdong,China,23.341700,113.424400,2020-07-09,1647,8,1631,8,Western Pacific
51311,,Libya,26.335100,17.228331,2020-08-01,3691,80,619,2992,Eastern Mediterranean
47440,Faroe Islands,Denmark,61.892600,-6.911800,2020-07-18,188,0,188,0,Europe


# Saving final data

In [410]:
# save data
full_table.to_csv('covid_19_clean_complete.csv', index=False)

# Full Grouped

In [411]:
# Grouped by day, country
# =======================

full_grouped = full_table.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

# new cases ======================================================

temp = full_grouped.groupby(['Country/Region', 'Date', ])['Confirmed', 'Deaths', 'Recovered']
temp.tail(10)


/Users/prathijathiruppathy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  after removing the cwd from sys.path.
/Users/prathijathiruppathy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Confirmed,Deaths,Recovered
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
37769,14510,100,8045
37770,10,1,8
37771,1832,518,915
37772,8210,241,6802


In [412]:
#percentage change

pc = temp.sum().diff().pct_change()*100
pc = pc.reset_index()
pc.columns = ['Country/Region', 'Date','% diff new cases', '% diff new deaths', '% diff new recovered']
pc.tail(10)

,Country/Region,Date,% diff new cases,% diff new deaths,% diff new recovered
37764,Zimbabwe,2020-08-01,536.363636,-85.714286,-91.250000
37765,Zimbabwe,2020-08-02,-46.530612,-50.000000,-28.571429
37766,Zimbabwe,2020-08-03,-41.221374,900.000000,720.000000
37767,Zimbabwe,2020-08-04,-5.194805,-90.000000,341.463415
37768,Zimbabwe,2020-08-05,-100.000000,-100.000000,-100.000000
37769,Zimbabwe,2020-08-06,inf,inf,inf
37770,Zimbabwe,2020-08-07,-5.084746,500.000000,211.538462
37771,Zimbabwe,2020-08-08,10.714286,-100.000000,-12.345679
37772,Zimbabwe,2020-08-09,-40.322581,inf,-70.422535
37773,Zimbabwe,2020-08-10,33.783784,-100.000000,314.285714


In [413]:
temp = temp.sum().diff().reset_index()

temp.head(10)



,Country/Region,Date,Confirmed,Deaths,Recovered
0,Afghanistan,2020-01-22,NaN,NaN,NaN
1,Afghanistan,2020-01-23,0.0,0.0,0.0
2,Afghanistan,2020-01-24,0.0,0.0,0.0
3,Afghanistan,2020-01-25,0.0,0.0,0.0
4,Afghanistan,2020-01-26,0.0,0.0,0.0
5,Afghanistan,2020-01-27,0.0,0.0,0.0
6,Afghanistan,2020-01-28,0.0,0.0,0.0
7,Afghanistan,2020-01-29,0.0,0.0,0.0
8,Afghanistan,2020-01-30,0.0,0.0,0.0
9,Afghanistan,2020-01-31,0.0,0.0,0.0


In [414]:
temp = pd.merge(temp, pc, on=['Country/Region', 'Date'])

temp.tail(10)

,Country/Region,Date,Confirmed,Deaths,Recovered,% diff new cases,% diff new deaths,% diff new recovered
37764,Zimbabwe,2020-08-01,490.0,2.0,7.0,536.363636,-85.714286,-91.250000
37765,Zimbabwe,2020-08-02,262.0,1.0,5.0,-46.530612,-50.000000,-28.571429
37766,Zimbabwe,2020-08-03,154.0,10.0,41.0,-41.221374,900.000000,720.000000
37767,Zimbabwe,2020-08-04,146.0,1.0,181.0,-5.194805,-90.000000,341.463415
37768,Zimbabwe,2020-08-05,0.0,0.0,0.0,-100.000000,-100.000000,-100.000000
37769,Zimbabwe,2020-08-06,118.0,3.0,26.0,inf,inf,inf
37770,Zimbabwe,2020-08-07,112.0,18.0,81.0,-5.084746,500.000000,211.538462
37771,Zimbabwe,2020-08-08,124.0,0.0,71.0,10.714286,-100.000000,-12.345679
37772,Zimbabwe,2020-08-09,74.0,2.0,21.0,-40.322581,inf,-70.422535
37773,Zimbabwe,2020-08-10,99.0,0.0,87.0,33.783784,-100.000000,314.285714


In [415]:
mask = temp['Country/Region'] != temp['Country/Region'].shift(1)

mask

0         True
1        False
2        False
3        False
4        False
         ...  
37769    False
37770    False
37771    False
37772    False
37773    False
Name: Country/Region, Length: 37774, dtype: bool

In [416]:
temp.loc[mask, 'Confirmed'] = np.nan
temp.loc[mask, 'Deaths'] = np.nan
temp.loc[mask, 'Recovered'] = np.nan
temp.loc[mask, '% diff new cases'] = np.nan
temp.loc[mask, '% diff new deaths'] = np.nan
temp.loc[mask, '% diff new recovered'] = np.nan

temp.tail(10)

,Country/Region,Date,Confirmed,Deaths,Recovered,% diff new cases,% diff new deaths,% diff new recovered
37764,Zimbabwe,2020-08-01,490.0,2.0,7.0,536.363636,-85.714286,-91.250000
37765,Zimbabwe,2020-08-02,262.0,1.0,5.0,-46.530612,-50.000000,-28.571429
37766,Zimbabwe,2020-08-03,154.0,10.0,41.0,-41.221374,900.000000,720.000000
37767,Zimbabwe,2020-08-04,146.0,1.0,181.0,-5.194805,-90.000000,341.463415
37768,Zimbabwe,2020-08-05,0.0,0.0,0.0,-100.000000,-100.000000,-100.000000
37769,Zimbabwe,2020-08-06,118.0,3.0,26.0,inf,inf,inf
37770,Zimbabwe,2020-08-07,112.0,18.0,81.0,-5.084746,500.000000,211.538462
37771,Zimbabwe,2020-08-08,124.0,0.0,71.0,10.714286,-100.000000,-12.345679
37772,Zimbabwe,2020-08-09,74.0,2.0,21.0,-40.322581,inf,-70.422535
37773,Zimbabwe,2020-08-10,99.0,0.0,87.0,33.783784,-100.000000,314.285714


In [417]:


# renaming columns
temp.columns = ['Country/Region', 'Date', 'New cases', 'New deaths', 'New recovered','% diff new cases', '% diff new deaths', '% diff new recovered']
# =================================================================
temp.tail(10)

,Country/Region,Date,New cases,New deaths,New recovered,% diff new cases,% diff new deaths,% diff new recovered
37764,Zimbabwe,2020-08-01,490.0,2.0,7.0,536.363636,-85.714286,-91.250000
37765,Zimbabwe,2020-08-02,262.0,1.0,5.0,-46.530612,-50.000000,-28.571429
37766,Zimbabwe,2020-08-03,154.0,10.0,41.0,-41.221374,900.000000,720.000000
37767,Zimbabwe,2020-08-04,146.0,1.0,181.0,-5.194805,-90.000000,341.463415
37768,Zimbabwe,2020-08-05,0.0,0.0,0.0,-100.000000,-100.000000,-100.000000
37769,Zimbabwe,2020-08-06,118.0,3.0,26.0,inf,inf,inf
37770,Zimbabwe,2020-08-07,112.0,18.0,81.0,-5.084746,500.000000,211.538462
37771,Zimbabwe,2020-08-08,124.0,0.0,71.0,10.714286,-100.000000,-12.345679
37772,Zimbabwe,2020-08-09,74.0,2.0,21.0,-40.322581,inf,-70.422535
37773,Zimbabwe,2020-08-10,99.0,0.0,87.0,33.783784,-100.000000,314.285714


In [418]:

# merging new values
full_grouped = pd.merge(full_grouped, temp, on=['Country/Region', 'Date'])

# filling na with 0
full_grouped = full_grouped.fillna(0)

# fixing data types
cols = ['New cases', 'New deaths', 'New recovered']
full_grouped[cols] = full_grouped[cols].astype('int')

full_grouped['New cases'] = full_grouped['New cases'].apply(lambda x: 0 if x<0 else x)

full_grouped.tail()

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,% diff new cases,% diff new deaths,% diff new recovered
37769,2020-08-10,West Bank and Gaza,14510,100,8045,6365,302,3,100,7.857143,200.00,-58.158996
37770,2020-08-10,Western Sahara,10,1,8,1,0,0,0,0.000000,0.00,0.000000
37771,2020-08-10,Yemen,1832,518,915,399,28,3,2,300.000000,0.00,-33.333333
37772,2020-08-10,Zambia,8210,241,6802,1167,125,6,104,-31.318681,-81.25,-61.048689
37773,2020-08-10,Zimbabwe,4748,104,1524,3120,99,0,87,33.783784,-100.00,314.285714


In [419]:
# add 'WHO Region' column
full_grouped['WHO Region'] = full_grouped['Country/Region'].map(who_region)

# find missing values
full_grouped[full_grouped['WHO Region'].isna()]['Country/Region'].unique()

array([], dtype=object)

In [420]:
#adding percentage difference for confirmed,deaths, recovered,active #changes made to obtain quick stats on covid project


#percentage difference in confirmed cases

#(252337- 234639)/234639*100 > newcaseslastday - newcaseslastbutbeforeday/newcaseslastbutbeforeday*100


#full_grouped['new cases % diff'] = full_grouped['']


In [421]:
# save as .csv file
full_grouped.to_csv('full_grouped.csv', index=False)

# Day wise

In [422]:
# Day wise
# ========

# table
day_wise = full_grouped.groupby('Date')['Confirmed', 'Deaths', 'Recovered', 
                                        'Active', 'New cases', 'New deaths', 'New recovered'].sum().reset_index()

# number cases per 100 cases
day_wise['Deaths / 100 Cases'] = round((day_wise['Deaths']/day_wise['Confirmed'])*100, 2)
day_wise['Recovered / 100 Cases'] = round((day_wise['Recovered']/day_wise['Confirmed'])*100, 2)
day_wise['Deaths / 100 Recovered'] = round((day_wise['Deaths']/day_wise['Recovered'])*100, 2)

# no. of countries
day_wise['No. of countries'] = full_grouped[full_grouped['Confirmed']!=0] \
                                    .groupby('Date')['Country/Region'] \
                                    .unique() \
                                    .apply(len)\
                                    .values

# fillna by 0
cols = ['Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered']
day_wise[cols] = day_wise[cols].fillna(0)

day_wise.head()

/Users/prathijathiruppathy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,2020-01-22,555,17,28,510,0,0,0,3.06,5.05,60.71,6
1,2020-01-23,654,18,30,606,99,1,2,2.75,4.59,60.00,8
2,2020-01-24,941,26,36,879,287,8,6,2.76,3.83,72.22,9
3,2020-01-25,1434,42,39,1353,493,16,3,2.93,2.72,107.69,11
4,2020-01-26,2118,56,52,2010,684,14,13,2.64,2.46,107.69,13


In [423]:
# save as .csv file
day_wise.to_csv('day_wise.csv', index=False)

# Country wise latest

In [424]:
# Country wise
# ============

full_grouped['Date'] = pd.to_datetime(full_grouped['Date'])

# getting latest values
country_wise = full_grouped[full_grouped['Date']==max(full_grouped['Date'])] \
                    .reset_index(drop=True) \
                    .drop('Date', axis=1)

print(country_wise.shape)

# group by country
country_wise = country_wise.groupby('Country/Region')['Confirmed', 'Deaths', 
                                                      'Recovered', 'Active', 
                                                      'New cases', 'New deaths', 'New recovered'].sum().reset_index()
print(country_wise.shape)


# per 100 cases
country_wise['Deaths / 100 Cases'] = round((country_wise['Deaths']/country_wise['Confirmed'])*100, 2)
country_wise['Recovered / 100 Cases'] = round((country_wise['Recovered']/country_wise['Confirmed'])*100, 2)
country_wise['Deaths / 100 Recovered'] = round((country_wise['Deaths']/country_wise['Recovered'])*100, 2)

cols = ['Deaths / 100 Cases', 'Recovered / 100 Cases', 'Deaths / 100 Recovered']
country_wise[cols] = country_wise[cols].fillna(0)


# 1 week increase and % change
# ============================

today = full_grouped[full_grouped['Date']==max(full_grouped['Date'])] \
            .reset_index(drop=True) \
            .drop('Date', axis=1)[['Country/Region', 'Confirmed']]

last_week = full_grouped[full_grouped['Date']==max(full_grouped['Date'])-timedelta(days=7)] \
                .reset_index(drop=True) \
                .drop('Date', axis=1)[['Country/Region', 'Confirmed']]

temp = pd.merge(today, last_week, on='Country/Region', suffixes=(' today', ' last week'))
temp['1 week change'] = temp['Confirmed today'] - temp['Confirmed last week']
temp = temp[['Country/Region', 'Confirmed last week', '1 week change']]

country_wise = pd.merge(country_wise, temp, on='Country/Region')
country_wise['1 week % increase'] = round(country_wise['1 week change']/country_wise['Confirmed last week']*100, 2)
country_wise.head()

country_wise['WHO Region'] = country_wise['Country/Region'].map(who_region)
country_wise[country_wise['WHO Region'].isna()]['Country/Region'].unique()

country_wise.head()

(187, 12)
(187, 8)


/Users/prathijathiruppathy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,Confirmed last week,1 week change,1 week % increase,WHO Region
0,Afghanistan,37162,1328,26228,9606,108,16,268,3.57,70.58,5.06,36747,415,1.13,Eastern Mediterranean
1,Albania,6536,200,3379,2957,125,1,37,3.06,51.70,5.92,5620,916,16.30,Europe
2,Algeria,35712,1312,24920,9480,552,10,414,3.67,69.78,5.26,31972,3740,11.70,Africa
3,Andorra,963,52,839,72,8,0,0,5.40,87.12,6.20,937,26,2.77,Europe
4,Angola,1679,78,569,1032,7,3,2,4.65,33.89,13.71,1280,399,31.17,Africa


In [425]:
# save as .csv file
country_wise.to_csv('country_wise_latest.csv', index=False)

# Country wise data

In [426]:
# # china
# # =====

# china_province_wise = full_table[full_table['Country/Region']=='China']
# china_province_wise['Province/State'].unique()
# china_province_wise.to_csv('china_province_wise.csv', index=False)

In [427]:
# # Australia
# # =========

# australia_state_wise = full_table[full_table['Country/Region']=='Australia']
# australia_state_wise['Province/State'].unique()
# australia_state_wise.to_csv('australia_state_wise.csv', index=False)

In [428]:
# # Canada
# # ======

# canada_state_wise = full_table[full_table['Country/Region']=='Canada']
# canada_state_wise['Province/State'].unique()
# canada_state_wise.to_csv('canada_state_wise.csv', index=False)

# USA data

In [429]:
# download data
# =============

# urls of the files
urls = ['https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv', 
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv']

# download files
for url in urls:
    filename = wget.download(url)

In [430]:
# read data
# =========

us_conf_df = pd.read_csv('time_series_covid19_confirmed_US.csv')
us_deaths_df = pd.read_csv('time_series_covid19_deaths_US.csv')

In [431]:
# ids
ids = us_conf_df.columns[0:11]
# dates
us_dates = us_conf_df.columns[11:]

# melt to longer format
us_conf_df_long = us_conf_df.melt(id_vars=ids, value_vars=us_dates, var_name='Date', value_name='Confirmed')
us_deaths_df_long = us_deaths_df.melt(id_vars=ids, value_vars=us_dates, var_name='Date', value_name='Deaths')

In [432]:
# first few rows
us_conf_df_long.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",1/22/20,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",1/22/20,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",1/22/20,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",1/22/20,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",1/22/20,0


In [433]:
# concatenate dataframes
us_full_table = pd.concat([us_conf_df_long, us_deaths_df_long[['Deaths']]], axis=1)

#first few rows
us_full_table.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",1/22/20,0,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",1/22/20,0,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",1/22/20,0,0


In [434]:
# save as .csv file
us_full_table.to_csv('usa_county_wise.csv', index=False)